# Function to complete exsting JSON files with isochrones data
Isochrones are from from https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post  

# **Warning: to run this notebook you need to have a OpenRouteService API key!**

# Dependencies

In [46]:
import json
from openrouteservice import client as ors
import yaml
import os
import pandas as pd
import plotly.graph_objects as go

# Load JSON files to be updated

In [49]:
def load_data(path):
    try: 
        with open(path, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error as occured: {e}")
 
def get_file_names(directory):
    return [
        os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')
    ]

In [50]:
directory = r"../data/google_data_isochrone_pop_cgpt"
FILES = get_file_names(directory)
FILES

['../data/google_data_isochrone_pop_cgpt\\Ex1_8004_Zurich_Werdgartengasse_4.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex2_3027_Bern_Colombstrasse_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex3_1006_Lausanne_Av_d_Ouchy_58.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex4_8355_Aadorf_Bruggwiesenstrasse_5.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex5_6319_Allenwinden_Winzruti_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex6_8005_Zurich_Heinrichstrasse_200.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex7_8003_Zurich_Birmensdorferstrasse_108.json']

# Function to add isochrone data

In [ ]:
def add_isochrone_data_to_existing_JSONfile(path):
    """Adds isochrones from openrouteservice.org to an existing json file.
    Does not do anything if the file already contains the isochrone data.
    Arg:
    - path : path incl. extension ".json"
    """
    # loads the file
    with open(path, 'r') as file:
        data = json.load(file)
    
        if 'isochrone' in data.keys():
            print(path +' : \nThis JSON file already has isochrones.')
        else:
            # Credenttials for API request:
            with open('credentials.json', 'r') as file:
                api_creds = yaml.safe_load(file)
            client = ors.Client(key=api_creds['ors_key'])
            # Makes the API request. Warning : here, FIRST lon and THEN lat !
            isochrone = client.isochrones(
                    locations=[  [data['original_address']['coordinates'][1], data['original_address']['coordinates'][0]]  ], 
                    profile="foot-walking", 
                    range=[n*60 for n in [3,5,7,10,13,15]],  # range has to be given in seconds
                    smoothing=0, 
                    range_type='time')
            # appends the isochrone data to the existing dictionary:
            data['isochrone'] = isochrone
            # overwrites the file:
            with open(path, 'w') as file:
                json.dump(data, file, indent=4)

# Update all files

In [51]:
for file in FILES:
    add_isochrone_data_to_existing_JSONfile(file)

../data/google_data_isochrone_pop_cgpt\Ex1_8004_Zurich_Werdgartengasse_4.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex2_3027_Bern_Colombstrasse_39.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex3_1006_Lausanne_Av_d_Ouchy_58.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex4_8355_Aadorf_Bruggwiesenstrasse_5.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex5_6319_Allenwinden_Winzruti_39.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex6_8005_Zurich_Heinrichstrasse_200.json : 
This JSON file already has isochrones.
../data/google_data_isochrone_pop_cgpt\Ex7_8003_Zurich_Birmensdorferstrasse_108.json : 
This JSON file already has isochrones.


# Adapt existing plotting code to take into account the new structure of the files

In [58]:
example = load_data(FILES[0])
isochrone = example['isochrone']

len(isochrone['features'])
isochrone['features'][2].keys()

dict_keys(['type', 'properties', 'geometry'])

In [59]:
# extract the coordinates out of the isochrone object
coords = isochrone['features'][0]['geometry']['coordinates'][0]
isochrone_df = pd.DataFrame( [{'lon': coord[0], 'lat': coord[1]} for coord in coords] )
print(coords)
isochrone_df

[[8.524168, 47.370571], [8.524235, 47.370183], [8.524484, 47.369923], [8.525599, 47.369475], [8.525857, 47.369446], [8.528416, 47.369476], [8.528616, 47.369775], [8.528689, 47.371224], [8.528622, 47.371502], [8.528074, 47.372038], [8.52724, 47.372559], [8.526745, 47.372665], [8.525476, 47.37269], [8.525261, 47.3727], [8.525194, 47.372675], [8.524173, 47.37082], [8.524168, 47.370571]]


,lon,lat
0,8.524168,47.370571
1,8.524235,47.370183
2,8.524484,47.369923
3,8.525599,47.369475
4,8.525857,47.369446
5,8.528416,47.369476
6,8.528616,47.369775
7,8.528689,47.371224
8,8.528622,47.371502
9,8.528074,47.372038


### create_base_map()

In [60]:
def create_base_map(FILE, width=800, height=600, zoom=13):

    # coords of original address:
    LAT = FILE['original_address']['coordinates'][0]
    LON = FILE['original_address']['coordinates'][1]

    # create base map with original adress.
    base_map = go.Figure(go.Scattermapbox())
    
    # Set up the layout for the base map
    base_map.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=zoom, 
        mapbox_center={"lat": LAT, "lon": LON},
        width=width,
        height=height,
        margin=dict(r=0, t=0, l=0, b=0), 
        showlegend=False
    )
    return base_map

### add_isochrone()

In [61]:
def get_isochrone_data(data):
    isochrone_data = []
    for isochrone in range(len(data['isochrone']['features'])):
        for coord in range(len(data['isochrone']['features'][isochrone]['geometry']['coordinates'][0])): #("[0]" at the end is necessary since there are two [] in excess!)
            isochrone_data.append({
                'travel_time': data['isochrone']['features'][isochrone]['properties']['value'],
                'lat': data['isochrone']['features'][isochrone]['geometry']['coordinates'][0][coord][1],
                'lon': data['isochrone']['features'][isochrone]['geometry']['coordinates'][0][coord][0],
            })
    df = pd.DataFrame(isochrone_data)
    return df
    
def add_isochrone(base_map, FILE):
    # format data
    df = get_isochrone_data(FILE)

    for travel_time in df['travel_time'].unique():
        df_filtered = df[df['travel_time']==travel_time]
        # create the layer:
        new_layer = go.Scattermapbox(
            lat=df_filtered['lat'],  
            lon=df_filtered['lon'],  
            mode='lines', # no markers
            line=dict(width=0),  # deletes line so as to display filling only
            fill='toself', 
            fillcolor='rgba(10,80,50,0.25)',  
            showlegend = False,
            marker=dict(size=1, color='blue'),
            text= str(travel_time/60) + 'min. walking',
            hoverinfo='text'
            )
        base_map.add_trace(new_layer)
        
    base_map.update_layout( margin=dict(r=0, t=0, l=0, b=0), showlegend=False)

    return base_map

### Try functions out

In [62]:
FILE = load_data(FILES[0])
base = create_base_map(FILE, width=800, height=600, zoom=13)
add_isochrone(base, FILE)

### OLD version  of the add_isochrone() function: the isochrone data was not yet IN THE JSON FILE , they were requested each time that the map was created, which is not very useful!

In [63]:
def add_isochrone_OLD(base_map, range_minutes, FILE):

    # parameters for API request. Warning : here, FIRST lon and THEN lat !
    coordinates= [  [FILE['original_address']['coordinates'][1], FILE['original_address']['coordinates'][0]]  ]
    profile="foot-walking"
    range_m=[range_minutes*60]  # range in seconds
    smoothing=0
    range_type='time'

    # API request
    isochrone = client.isochrones(locations=coordinates, 
                        profile=profile, 
                        range=range_m, 
                        smoothing=smoothing, 
                        range_type=range_type)

    # extract the coordinates out of the isochrone object
    coords = isochrone['features'][0]['geometry']['coordinates'][0]
    isochrone_df = pd.DataFrame( [{'lon': coord[0], 'lat': coord[1]} for coord in coords] )
    
    # create the layer:
    new_layer = go.Scattermapbox(
        lat=isochrone_df['lat'],  
        lon=isochrone_df['lon'],  
        mode='lines', # no markers
        line=dict(width=0),  # delete line only display the filling
        fill='toself', 
        fillcolor='rgba(0,50,80,0.25)',  
        showlegend = False,
        marker=dict(size=1, color='blue'),
        text= str(range_minutes) + 'min. walking',
        hoverinfo='text'
        # hovertemplate='%{text}<extra></extra>'  
    )

    base_map.add_trace(new_layer)
    base_map.update_layout( margin=dict(r=0, t=0, l=0, b=0), showlegend=False)

    return base_map

In [64]:
FILE = load_data(FILES[0])

with open('credentials.json', 'r') as file:
    api_creds = yaml.safe_load(file)
client = ors.Client(key=api_creds['ors_key'])

base = create_base_map(FILE, width=800, height=600, zoom=13)
add_isochrone_OLD(base, 15, FILE)